In [4]:
from pyspark.mllib.fpm import FPGrowth
from pyspark.mllib.fpm import PrefixSpan

In [8]:
domain_data = sc.textFile('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data/part-*')
domain_data.cache
num_flow = domain_data.count()

In [9]:
minSupport = 0.14
print(">>",num_flow*minSupport,"will show")

transactions = domain_data.map(lambda line: set(line.strip().split(' ')))
model = FPGrowth.train(transactions, minSupport=minSupport, numPartitions=10)
FPGrowth_result = model.freqItemsets().collect()
for fi in FPGrowth_result[:10000]:
    if len(fi[0]) > 4:
        print(fi)

>> 398237.28 will show


In [10]:
len(FPGrowth_result)

133

In [169]:
minSupport = 0.2
print(">>",num_flow*minSupport,"will show")

transactions = domain_data.map(lambda line: set(line.strip().split(' ')))
model = FPGrowth.train(transactions, minSupport=minSupport, numPartitions=10)
FPGrowth_result = model.freqItemsets().collect()
FPGrowth_result

[FreqItemset(items=['wx.qlogo.cn'], freq=690463),
 FreqItemset(items=['wx.qlogo.cn', 'mmsns.qpic.cn'], freq=576429),
 FreqItemset(items=['wx.qlogo.cn', 'short.weixin.qq.com'], freq=646246),
 FreqItemset(items=['monitor.uu.qq.com'], freq=1358433),
 FreqItemset(items=['qzonestyle.gtimg.cn'], freq=593013),
 FreqItemset(items=['short.weixin.qq.com'], freq=1354793),
 FreqItemset(items=['short.weixin.qq.com', 'monitor.uu.qq.com'], freq=857050),
 FreqItemset(items=['configsvr.msf.3g.qq.com'], freq=1149171),
 FreqItemset(items=['configsvr.msf.3g.qq.com', 'short.weixin.qq.com'], freq=727529),
 FreqItemset(items=['configsvr.msf.3g.qq.com', 'short.weixin.qq.com', 'monitor.uu.qq.com'], freq=654573),
 FreqItemset(items=['configsvr.msf.3g.qq.com', 'monitor.uu.qq.com'], freq=1023811),
 FreqItemset(items=['dns.weixin.qq.com'], freq=1094191),
 FreqItemset(items=['dns.weixin.qq.com', 'configsvr.msf.3g.qq.com'], freq=625692),
 FreqItemset(items=['dns.weixin.qq.com', 'configsvr.msf.3g.qq.com', 'short.weix

In [12]:
a = "conf.3g.qq.com plus.xiangji.qq.com conf.3g.qq.com plus.xiangji.qq.com fr-talk.kakao.com plus-talk.kakao.com fr-talk.kakao.com st-talk.kakao.com monitor.uu.qq.com st-talk.kakao.com plus.xiangji.qq.com conf.3g.qq.com plus.xiangji.qq.com"
def constract_list_seq(str):
    i = 0
    n = 0
    origin = a.split(" ")
#     print(origin)
    result = [[origin[0]]]
    for j in range(1, len(origin)):
    #     print("------>>>",i,j,n)
        if origin[j].split(".")[-2] != origin[i].split(".")[-2]:
            tmp = []
            tmp.append(origin[j])
            result.append(tmp)
            i = j
    #         print(result)
            n = n + 1
        else:
            if len(result) == n+1:
    #             print(result)
                result[n].append(origin[j])
            else:
                tmp = []
                tmp.append(origin[i])
                result.append(tmp)
    #             print(origin[i])
    #             print(tmp)
    #             print(result)
    return result
print(constract_list_seq(a))

[['conf.3g.qq.com', 'plus.xiangji.qq.com', 'conf.3g.qq.com', 'plus.xiangji.qq.com'], ['fr-talk.kakao.com', 'plus-talk.kakao.com', 'fr-talk.kakao.com', 'st-talk.kakao.com'], ['monitor.uu.qq.com'], ['st-talk.kakao.com'], ['plus.xiangji.qq.com', 'conf.3g.qq.com', 'plus.xiangji.qq.com']]


In [13]:
transactions = domain_data.map(lambda line: constract_list_seq(line))
model = PrefixSpan.train(transactions)
PrefixSpan_result = model.freqSequences().collect()
for fi in PrefixSpan_result[:1000]:
    if len(fi[0]) > 10 and fi[1] >= 0.4*num_flow:
        print(fi)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 14 in stage 29.0 failed 1 times, most recent failure: Lost task 14.0 in stage 29.0 (TID 387182, localhost): java.lang.OutOfMemoryError: GC overhead limit exceeded

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1454)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1442)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1441)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1441)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$